In [1]:
# --- CELDA 1: INSTALACIÓN SEGURA (CPU MODE) ---

# 1. Instalar SOLO transformers y datasets.
# IMPORTANTE: No ponemos "-U" (upgrade) general para no romper torch.
!pip install transformers datasets

import os
import sys
import torch

print(f"🔧 Estado del Sistema:")
print(f"   Python: {sys.version.split()[0]}")
print(f"   Torch (Nativo): {torch.__version__}")

# 2. Intentar importar Transformers
try:
    from transformers import GPT2LMHeadModel, GPT2Tokenizer
    print("✅ Transformers cargado EXITOSAMENTE.")
except ImportError as e:
    print("⚠️ Hubo un problema. Intentando fix de emergencia...")
    !pip install transformers --force-reinstall --no-deps
    from transformers import GPT2LMHeadModel, GPT2Tokenizer
    print("✅ Transformers recuperado.")

print("🚀 LISTO PARA INFERENCIA.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 36.3 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12==25.6.*, but you have pylibcudf-cu12 25.2.2 which is incompatible.


2025-12-14 19:53:14.919524: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765741995.419554      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765741995.548603      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

✅ Transformers cargado EXITOSAMENTE.
🚀 LISTO PARA INFERENCIA.


In [2]:
import os

# 1. Definimos la ruta base exacta según tu captura de pantalla
# Kaggle monta el dataset con el nombre-del-dataset normalizado (guiones en vez de espacios)
DATASET_ROOT = "/kaggle/input/ascii-architect-v1"
SUB_FOLDER = "ASCII_Architect_V1_Models"

# Construimos la ruta absoluta
MODELS_DIR = os.path.join(DATASET_ROOT, SUB_FOLDER)

# 2. Definimos los expertos
MODEL_PATHS = {
    "BOX": os.path.join(MODELS_DIR, "expert_box"),
    "ARROW": os.path.join(MODELS_DIR, "expert_arrow")
}

# 3. Verificación Rápida (Solo para estar 100% seguros)
print(f"🎯 Apuntando a: {MODELS_DIR}")

all_found = True
for name, path in MODEL_PATHS.items():
    if os.path.exists(path):
        print(f"   ✅ {name}: Listo en {path}")
    else:
        print(f"   ❌ {name}: NO encontrado. Revisando contenido de carpeta raíz...")
        try:
            print(os.listdir(DATASET_ROOT))
        except:
            print("No pude leer el directorio.")
        all_found = False

if all_found:
    print("\n🚀 RUTAS CONFIRMADAS. Pasando al Motor.")

🎯 Apuntando a: /kaggle/input/ascii-architect-v1/ASCII_Architect_V1_Models
   ✅ BOX: Listo en /kaggle/input/ascii-architect-v1/ASCII_Architect_V1_Models/expert_box
   ✅ ARROW: Listo en /kaggle/input/ascii-architect-v1/ASCII_Architect_V1_Models/expert_arrow

🚀 RUTAS CONFIRMADAS. Pasando al Motor.


In [3]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import re

class ArchitectEngine:
    def __init__(self, paths):
        self.paths = paths
        self.device = "cpu" # Explícito para que no busque CUDA
        print("🔧 Motor Arquitecto V18 arrancando en CPU...")

    def generate(self, expert_type, tags, metadata):
        model_path = self.paths.get(expert_type)
        if not model_path: return "Error: Path desconocido"

        try:
            # Carga bajo demanda (Carga -> Genera -> Descarga) para ahorrar RAM
            tokenizer = GPT2Tokenizer.from_pretrained(model_path, local_files_only=True)
            model = GPT2LMHeadModel.from_pretrained(model_path, local_files_only=True)
            
            prompt = f"[TYPE:{expert_type}] {tags} {metadata}"
            inputs = tokenizer(prompt, return_tensors="pt")

            # Generación
            out = model.generate(
                **inputs, 
                max_length=120, # Suficiente para cajas medianas
                pad_token_id=50256, 
                do_sample=False # Determinista = Más rápido y preciso
            )
            raw = tokenizer.decode(out[0])
            
            # Limpieza V18
            return self._clean(raw, prompt)

        except Exception as e:
            return f"Error: {e}"

    def _clean(self, raw, prompt):
        # 1. Guillotina
        content = raw.split("[STOP]")[0]
        # 2. Extracción <L01>
        if "<L01>" in content:
            content = "<L01>" + content.split("<L01>", 1)[1]
        else:
            content = content.replace(prompt, "")
        
        # 3. Limpieza cosmética
        content = content.replace(" ", "")   # Quitar espacios del tokenizer
        ascii_art = content.replace("░", " ") # Transparencia -> Espacio
        
        # 4. Borrar metadatos
        ascii_art = re.sub(r'<L\d+>', '', ascii_art) 
        ascii_art = re.sub(r'\[S:\d+\]', '', ascii_art)
        
        return ascii_art.strip()

# Instanciar
architect = ArchitectEngine(MODEL_PATHS)

🔧 Motor Arquitecto V18 arrancando en CPU...


In [4]:
print("\n============== 🏗️ PRUEBAS DE CAMPO V18 ==============")

# 1. CAJAS (El Albañil)
print("\n📦 TEST 1: Caja Sólida (10x5)")
print(architect.generate("BOX", "[STYLE:SOLID]", "[DIM:10x5]"))

print("\n📦 TEST 2: Servidor Grande (20x6)")
print(architect.generate("BOX", "[STYLE:SOLID]", "[DIM:20x6]"))

# 2. FLECHAS (El Fontanero)
print("\n➡️ TEST 3: Flujo de Datos (Derecha Largo 8)")
print(architect.generate("ARROW", "[DIR:RIGHT]", "[LEN:8]"))

print("\n⬇️ TEST 4: Conexión Vertical (Abajo Largo 4)")
print(architect.generate("ARROW", "[DIR:DOWN]", "[LEN:4]"))

print("\n====================================================")


============== 🏗️ PRUEBAS DE CAMPO V18 ==============

📦 TEST 1: Caja Sólida (10x5)
+----N---+
|        |
W        E
|        |
+----S---+

📦 TEST 2: Servidor Grande (20x6)
+---------N--------+
|                  |
W                  E
|                  |
|

➡️ TEST 3: Flujo de Datos (Derecha Largo 8)
------->

⬇️ TEST 4: Conexión Vertical (Abajo Largo 4)
|
|
|
v



In [5]:
print("\n🔥 INICIANDO STRESS TEST: 'EL MORBO' 🔥")

# 1. EL RASCACIELOS (Verticality Stress)
# Reto: Mantener las paredes rectas durante muchas líneas sin que se doblen.
print("\n🏙️ TEST EXTREMO 1: El Rascacielos (7x15)")
# Nota: Es estrecho pero muy alto.
print(architect.generate("BOX", "[STYLE:SOLID]", "[DIM:7x15]"))

# 2. LA AUTOPISTA (Horizontal Stress)
# Reto: Una caja muy ancha y baja. ¿Se cansará de poner guiones?
print("\n🚛 TEST EXTREMO 2: La Autopista (35x3)")
print(architect.generate("BOX", "[STYLE:SOLID]", "[DIM:35x3]"))

# 3. FÍSICA CUÁNTICA (Micro-Box)
# Reto: Una caja tan pequeña que apenas caben los anchors.
# 4 de ancho (Borde + Espacio + Espacio + Borde)
print("\n🔬 TEST EXTREMO 3: La Partícula (4x3)")
print(architect.generate("BOX", "[STYLE:SOLID]", "[DIM:4x3]"))

# 4. LA LANZA DE LONGINUS (Arrow Length)
# Reto: Una flecha ridículamente larga hacia la izquierda.
print("\n⬅️ TEST EXTREMO 4: La Lanza Infinita (Left 25)")
print(architect.generate("ARROW", "[DIR:LEFT]", "[LEN:25]"))

# 5. EL CUADRADO PERFECTO (Symmetry)
# Reto: Ver si el aspecto visual es cuadrado (aunque en texto 1 caracter de alto != 1 de ancho)
print("\n⬜ TEST EXTREMO 5: El Cubo (12x12)")
print(architect.generate("BOX", "[STYLE:SOLID]", "[DIM:12x12]"))

print("\n🔥 FIN DE LA TORTURA 🔥")


🔥 INICIANDO STRESS TEST: 'EL MORBO' 🔥

🏙️ TEST EXTREMO 1: El Rascacielos (7x15)
+--N--+
|     |
|     |
W     E
|     |
|     |
|     |
+--S--+

🚛 TEST EXTREMO 2: La Autopista (35x3)
+------N------+
W             E
+------S------+

🔬 TEST EXTREMO 3: La Partícula (4x3)
+--N-+
W    E
+--S-+

⬅️ TEST EXTREMO 4: La Lanza Infinita (Left 25)
<-------

⬜ TEST EXTREMO 5: El Cubo (12x12)
+-----N----+
|          |
|          |
W          E
|          |
|          |
|

🔥 FIN DE LA TORTURA 🔥
